In [1]:
import os
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.models import alexnet

In [2]:
model = alexnet()
convs = []
for i,k in enumerate(model.modules()):
    if isinstance(k, nn.Conv2d):
        convs.append(k)

In [3]:
l1 = convs[0].weight

In [6]:
for j in range(l1.shape[1]): # itera sobre cada canal
    X = None
    y = None
    for i,w in enumerate(l1): #itera sobre cada filtro
        if X == None:
            X = torch.reshape(w[j], [1,w[j].shape[1]**2])  # <----- Problema: repetindo 1o elemento
        else:
            y = torch.reshape(w[0], [1,w[j].shape[1]**2])
            X = torch.cat((X,y))
    dist = _mahalanobis(X)

In [7]:
x = X[0] #vector1
y = X[2] #vector 2
### mahalanobis params
VI = torch.inverse(cov(X)) #inverse of covariance matrix
p2 = (x - y).unsqueeze(0)
p1 = (x-y).unsqueeze(0).t()

In [8]:
p1.shape

torch.Size([121, 1])

In [9]:
p2.shape

torch.Size([1, 121])

In [49]:
torch.sqrt(torch.mm(p2,torch.mm(VI,p1))) # this?

tensor([[25.2247]], grad_fn=<SqrtBackward>)

In [46]:
torch.sqrt(torch.mm(torch.mm(p2,VI),p1)) # or this?

tensor([[29.5692]], grad_fn=<SqrtBackward>)

In [67]:
_mahalanobis(X)

tensor([[0.]], grad_fn=<SqrtBackward>)
tensor([[34.6697]], grad_fn=<SqrtBackward>)
tensor([[23.2548]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[9.3950]], grad_fn=<SqrtBackward>)
tensor([[17.6363]], grad_fn=<SqrtBackward>)
tensor([[17.2331]], grad_fn=<SqrtBackward>)
tensor([[29.2530]], grad_fn=<SqrtBackward>)
tensor([[26.1127]], grad_fn=<SqrtBackward>)
tensor([[14.4489]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[28.5416]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[25.7847]], grad_fn=<SqrtBackward>)
tensor([[22.2904]], grad_fn=<SqrtBackward>)
tensor([[5.2623]], grad_fn=<SqrtBackward>)
tensor([[14.6918]], grad_fn=<SqrtBackward>)
tensor([[29.7986]], grad_fn=<SqrtBackward>)
tensor([[17.1704]], grad_fn=<SqrtBackward>)
tensor([[20.5085]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[33.9384]],

tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[24.5316]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[9.9048]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[21.8460]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[14.9010]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[9.6935]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[25.4466]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[24.5153]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]]

tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[25.6745]], grad_fn=<SqrtBackward>)
tensor([[48.6128]], grad_fn=<SqrtBackward>)
tensor([[15.0376]], grad_fn=<SqrtBackward>)
tensor([[14.4489]], grad_fn=<SqrtBackward>)
tensor([[14.4489]], grad_fn=<SqrtBackward>)
tensor([[36.7338]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[24.7025]], grad_fn=<SqrtBackward>)
tensor([[4.8480]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[21.5889]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tenso

tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[13.9702]], grad_fn=<SqrtBackward>)
tensor([[6.1037]], grad_fn=<SqrtBackward>)
tensor([[4.2052]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[8.8389]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[22.0438]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[3.5729]], grad_fn=<SqrtBackward>)
tensor([[21.4663]], grad_fn=<SqrtBackward>)
tensor([[10.0028]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[24.9590]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[28.5040]], grad_fn=<SqrtBackward>)
tens

tensor([[27.4706]], grad_fn=<SqrtBackward>)
tensor([[12.4732]], grad_fn=<SqrtBackward>)
tensor([[17.3741]], grad_fn=<SqrtBackward>)
tensor([[17.4991]], grad_fn=<SqrtBackward>)
tensor([[33.7251]], grad_fn=<SqrtBackward>)
tensor([[28.4900]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[25.0561]], grad_fn=<SqrtBackward>)
tensor([[19.0509]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[32.3503]], grad_fn=<SqrtBackward>)
tensor([[15.6302]], grad_fn=<SqrtBackward>)
tensor([[27.6540]], grad_fn=<SqrtBackward>)
tensor([[13.7347]], grad_fn=<SqrtBackward>)
tensor([[27.6717]], grad_fn=<SqrtBackward>)
tensor([[35.4267]], grad_fn=<SqrtBackward>)
tensor([[21.0777]], grad_fn=<SqrtBackward>)
tensor([[33.6224]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[32.8462]], grad_fn=<SqrtBackward>)
tensor([[24.1770]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([

...

tensor([[14.0654]], grad_fn=<SqrtBackward>)
tensor([[26.0218]], grad_fn=<SqrtBackward>)
tensor([[28.0678]], grad_fn=<SqrtBackward>)
tensor([[33.3225]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[24.1757]], grad_fn=<SqrtBackward>)
tensor([[23.7877]], grad_fn=<SqrtBackward>)
tensor([[24.7098]], grad_fn=<SqrtBackward>)
tensor([[36.0611]], grad_fn=<SqrtBackward>)
tensor([[26.1493]], grad_fn=<SqrtBackward>)
tensor([[40.5901]], grad_fn=<SqrtBackward>)
tensor([[24.0835]], grad_fn=<SqrtBackward>)
tensor([[38.0877]], grad_fn=<SqrtBackward>)
tensor([[22.9358]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[24.4608]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[14.8351]], grad_fn=<SqrtBackward>)
tensor([[19.8800]], grad_fn=<SqrtBackward>)
tensor([[35.7456]], grad_fn=<SqrtBackward>)
tensor([[30.9720]], grad_fn=<SqrtBackward>)
tensor([[28.7988]], grad_fn=<SqrtBackward>)
tensor([[40.5612]], grad_fn=<SqrtBackward>)


tensor([[22.8403]], grad_fn=<SqrtBackward>)
tensor([[8.6165]], grad_fn=<SqrtBackward>)
tensor([[19.3603]], grad_fn=<SqrtBackward>)
tensor([[37.1550]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[26.6828]], grad_fn=<SqrtBackward>)
tensor([[15.3721]], grad_fn=<SqrtBackward>)
tensor([[18.4466]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[24.4295]], grad_fn=<SqrtBackward>)
tensor([[10.2807]], grad_fn=<SqrtBackward>)
tensor([[22.4490]], grad_fn=<SqrtBackward>)
tensor([[25.2661]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[30.8938]], grad_fn=<SqrtBackward>)
tensor([[15.9381]], grad_fn=<SqrtBackward>)
tensor([[20.9576]], grad_fn=<SqrtBackward>)
tensor([[20.9576]], grad_fn=<SqrtBackward>)
tensor([[35.3316]], grad_fn=<SqrtBackward>)
tensor([[38.5053]

tensor([[23.9130]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[20.7006]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[32.3543]], grad_fn=<SqrtBackward>)
tensor([[32.3543]], grad_fn=<SqrtBackward>)
tensor([[29.8398]], grad_fn=<SqrtBackward>)
tensor([[38.4428]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[13.1545]], grad_fn=<SqrtBackward>)
tensor([[36.1626]], grad_fn=<SqrtBackward>)
tensor([[12.0597]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[30.2585]], grad_fn=<SqrtBackward>)
tensor([[32.3348]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[39.8173]], grad_fn=<SqrtBackward>)
tensor([[46.8288]], grad_fn=<SqrtBackward>)
tensor([[nan]], grad_fn=<SqrtBackward>)
tensor([[28.5040]], grad_fn=<SqrtBackward>)
tensor([[37.0566]], grad_fn=<SqrtBackward>)
tensor([[12.9523]], grad_fn=<SqrtBackward>)
tensor([[16.4590]], grad_fn=<SqrtBackward>)
tensor([[12.

tensor([[28.0867]], grad_fn=<SqrtBackward>)
tensor([[34.8423]], grad_fn=<SqrtBackward>)
tensor([[31.6370]], grad_fn=<SqrtBackward>)
tensor([[41.2909]], grad_fn=<SqrtBackward>)
tensor([[47.5498]], grad_fn=<SqrtBackward>)
tensor([[34.4548]], grad_fn=<SqrtBackward>)
tensor([[38.3939]], grad_fn=<SqrtBackward>)
tensor([[49.6067]], grad_fn=<SqrtBackward>)
tensor([[25.9063]], grad_fn=<SqrtBackward>)
tensor([[39.0256]], grad_fn=<SqrtBackward>)
tensor([[33.4475]], grad_fn=<SqrtBackward>)
tensor([[27.6482]], grad_fn=<SqrtBackward>)
tensor([[42.2699]], grad_fn=<SqrtBackward>)
tensor([[46.7630]], grad_fn=<SqrtBackward>)
tensor([[31.1240]], grad_fn=<SqrtBackward>)
tensor([[20.2835]], grad_fn=<SqrtBackward>)
tensor([[22.2712]], grad_fn=<SqrtBackward>)
tensor([[26.0246]], grad_fn=<SqrtBackward>)
tensor([[24.8012]], grad_fn=<SqrtBackward>)
tensor([[21.2050]], grad_fn=<SqrtBackward>)
tensor([[48.3824]], grad_fn=<SqrtBackward>)
tensor([[28.5810]], grad_fn=<SqrtBackward>)
tensor([[27.5210]], grad_fn=<Sqr

tensor([[nan]], grad_fn=<ThAddBackward>)

In [66]:
def _mahalanobis(X):
    VI = torch.inverse(cov(X)) #covariance matrix
    total_dist = 0
    for i,v in enumerate(X):
        dist = 0
        for j,u in enumerate(X):
            if i == j:
                continue
            x = (v-u).unsqueeze(0).t()
            y = (v - u).unsqueeze(0)
            
            dist = torch.sqrt(torch.mm(torch.mm(y,VI),x))
            
            total_dist +=dist
            print(dist)
    return total_dist

In [19]:
range(l1.shape[0])

range(0, 64)

In [ ]:
S = torch.inverse(cov(w1))
mu = torch.mean(w1)

In [4]:
import torch

def cov(m, rowvar=False):
    '''Estimate a covariance matrix given data.

    Covariance indicates the level to which two variables vary together.
    If we examine N-dimensional samples, `X = [x_1, x_2, ... x_N]^T`,
    then the covariance matrix element `C_{ij}` is the covariance of
    `x_i` and `x_j`. The element `C_{ii}` is the variance of `x_i`.

    Args:
        m: A 1-D or 2-D array containing multiple variables and observations.
            Each row of `m` represents a variable, and each column a single
            observation of all those variables.
        rowvar: If `rowvar` is True, then each row represents a
            variable, with observations in the columns. Otherwise, the
            relationship is transposed: each column represents a variable,
            while the rows contain observations.

    Returns:
        The covariance matrix of the variables.
    '''
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    # m = m.type(torch.double)  # uncomment this line if desired
    fact = 1.0 / (m.size(1) - 1)
    m -= torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()